In [67]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error

In [2]:
from proc_v1 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter

In [9]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform(['data/train.csv']).assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform(['data/test.csv'])

In [5]:
df_train.columns

Index(['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp',
       'Calories', 'Duration_log', 'Heart_Rate_div_Weight_sqrt',
       'Age_div_Weight_sqrt', 'Body_Temp_div_Heart_Rate', 'Weight_2_Duration'],
      dtype='object')

# LGB 1

- Duration 대신에 Duration_log를 사용하고 나머지 변수를 넣습니다.

In [116]:
hparams = {
    'model_params': {'n_estimators': 2000, 'colsample_bytree': 0.9, 'num_leaves': 63, 'learning_rate': 0.03},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('lgb1', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.06019226677386045,
 [0.06007911150499369,
  0.06108536277081935,
  0.05890153017031521,
  0.060703062649313556])

# LGB 2

In [121]:
hparams = {
    'model_params': {'n_estimators': 2500, 'colsample_bytree': 0.9, 'num_leaves': 7, 'learning_rate': 0.07},
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
    #'validation_fraction': 0.1, 
}
result = sc.cv_result('lgb2', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.06020235343334941,
 [0.06012424687183828,
  0.06109689964224183,
  0.05889602519069495,
  0.060692242028622584])

In [122]:
root_mean_squared_error(
    df_train[target],
    sc.read_prds(['lgb2', 'lgb1'], index = df_train.index).dot([0.5, 0.5])
)

0.05982246533502571

# CB1

In [104]:
hparams = {
    'model_params': {'max_depth': 5, 'n_estimators': 2000, 'learning_rate': 0.1}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb1', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.059661819891972,
 [0.05942043391561231,
  0.060589339037000235,
  0.05830634124626922,
  0.06033116536900623])

# CB2

In [108]:
hparams = {
    'model_params': {'max_depth': 6, 'n_estimators': 2000, 'learning_rate': 0.1}, 
    'X_num': ['Sex', 'Age', 'Height', 'Weight', 'Duration_log', 'Heart_Rate', 'Body_Temp'], 
}

result = sc.cv_result('cb2', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result['valid_scores']), result['valid_scores']

Fold:   0%|          | 0/4 [00:00<?, ?it/s]

(0.05967605446748434,
 [0.05940975474268567,
  0.06056260252132592,
  0.05843785657515635,
  0.060294004030769434])

In [123]:
root_mean_squared_error(
    df_train[target],
    sc.read_prds(['lgb2', 'lgb1', 'cb1', 'cb2'], index = df_train.index).dot([0.1, 0.1, 0.4, 0.4])
)

0.05949587467354857